## import library yang akan digunakan

In [16]:
import pandas as pd
import numpy as np
import time
import argparse
import string
from sklearn.model_selection import train_test_split
from nltk.tokenize import regexp_tokenize
from datetime import datetime
import pytz

In [17]:
data_frame = pd.read_csv('hasil labeling data.csv')
data_frame.head()

,username,tanggal,tweet,sentimen
0,pkmjangkar,3/7/2022 23:58,update jadwal vaksinasi covid 19 upt puskesmas...,positif
1,CNNIndonesia,3/7/2022 23:57,syarat jalan polymerase chain reaction antigen...,netral
2,DindaNatasha15,3/7/2022 23:56,perintah pasti vaksin covid 19 bukti aman uji ...,positif
3,juvejack,3/7/2022 23:49,menko maritim investasi luhut pandjaitan tanga...,netral
4,nengsunshine,3/7/2022 23:47,ok google negatif covid vaksin,netral


In [18]:
#repalcing the categorical values of 'airline_sentiment' to numeric values
data_frame['sentimen'].replace(('negatif', 'netral', 'positif'), (0, 1, 2), inplace=True)
data_frame['sentimen'].value_counts()

1    645
2    250
0    157
Name: sentimen, dtype: int64

In [19]:
data = data_frame['tweet'].values.tolist()
labels = data_frame['sentimen'].values.tolist()

In [20]:
train_X, test_X, y_train, y_test = train_test_split(data, labels, test_size=0.2, 
                                                    random_state=0)


## Berikut adalah pola default untuk tokenisasi

In [21]:
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """

## funtion untuk tokenisasi data

In [22]:
def tokenize(text, pattern = default_pattern):
    text = text.lower()
    return regexp_tokenize(text, pattern)

## Tokenize train dan test text menjadi tokens

In [23]:
tokenized_text = []
for i in range(0, len(train_X)):
    tokenized_text.append(tokenize(train_X[i]))

X_train = tokenized_text


tokenized_text = []
for i in range(0, len(test_X)):
    tokenized_text.append(tokenize(test_X[i]))

X_test = tokenized_text

## Untuk membuat kamus token dari data
## data dalam tipe - daftar
## Kamus token yang diurutkan dan jumlahnya dalam data

In [24]:
def createDictionary(data):
    dictionary = dict()
    for sample in  data:
        for token in sample:
            dictionary[token] = dictionary.get(token, 0) + 1
    sorted_dict = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_dict)

In [25]:
class NBClassifier:

    def __init__(self, X_train, y_train, size):
        self.X_train = X_train
        self.y_train = y_train
        self.size = size

# Untuk membuat kamus token dari data
# data dalam tipe - daftar
# Kamus yang diurutkan dari token dan jumlah mereka dalam data
    def createDictionary(self):
        dictionary = dict()
        for sample in  X_train:
            for token in sample:
                dictionary[token] = dictionary.get(token, 0) + 1
        sorted_dict = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
        return dict(sorted_dict)
# Untuk menghitung jumlah kata dalam kamus data pelatihan
# Trianing data & Ukuran kamus
# kamus token dengan probabilitas bijaksana kelasnya 
    def fit(self):
      
        X_train_dict = self.createDictionary()
        if self.size == 'full':
            self.words_list = list(X_train_dict.keys())
            self.words_count = dict.fromkeys(self.words_list, None)
        else:
            self.words_list = list(X_train_dict.keys())[:int(self.size)]
            self.words_count = dict.fromkeys(self.words_list, None)
            
        train = pd.DataFrame(columns = ['X_train', 'y_train'])
        train['X_train'] = X_train
        train['y_train'] = y_train

        train_0 = train.copy()[train['y_train'] == 0]
        train_1 = train.copy()[train['y_train'] == 1]
        train_2 = train.copy()[train['y_train'] == 2]
        
        Pr0 = train_0.shape[0]/train.shape[0]
        Pr1 = train_1.shape[0]/train.shape[0]
        Pr2 = train_2.shape[0]/train.shape[0]

        self.Prior = np.array([Pr0, Pr1, Pr2])

        def flatList(listOfList):
            flatten = []
            for elem in listOfList:
                flatten.extend(elem)
            return flatten
 
        X_train_0 = flatList(train[train['y_train'] == 0]['X_train'].tolist())
        X_train_1 = flatList(train[train['y_train'] == 1]['X_train'].tolist())
        X_train_2 = flatList(train[train['y_train'] == 2]['X_train'].tolist())

        self.X_train_len = np.array([len(X_train_0), len(X_train_1), len(X_train_2)])

        for token in self.words_list:
          
            res = [] #list untuk menyimpan label

            res.insert(0, X_train_0.count(token))


            res.insert(1, X_train_1.count(token))

            res.insert(2, X_train_2.count(token))


            self.words_count[token] = res
        return self
# Memprediksi label data
# menghitung data uji
# Daftar label yang diprediksi untuk data uji
    def predict(self, X_test):
        pred = []
        for sample in X_test:
            mul = np.array([1,1,1])
            for tokens in sample:
                vocab_count = len(self.words_list)
                if tokens in self.words_list:
                    prob = ((np.array(self.words_count[tokens])+1) / (self.X_train_len + vocab_count))
                mul = mul * prob
            val = mul * self.Prior
            pred.append(np.argmax(val))
        return pred
# Untuk menghitung kinerja
# label yang diprediksi, dan label aktual dari data uji
# Jumlah label yang diprediksi dengan benar dan akurasi
    def score(self, pred, labels):

        correct = (np.array(pred) == np.array(labels)).sum()
        accuracy = correct/len(pred)
        return correct, accuracy

In [26]:
# Creating holders to store the model performance results
attributes = []
corr = []
acc = []

#function to call for storing the results
def storeResults(attr, cor,ac):
    attributes.append(attr)
    corr.append(round(cor, 3))
    acc.append(round(ac, 3))

In [27]:
#training the classifier     
nb = NBClassifier(X_train, y_train, 'full')  
nb.fit()
y_pred = nb.predict(X_test)

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("\nConfusion Matrix")
print(confusion_matrix(y_pred, y_test))

print("\nClassification Report")
print(classification_report(y_pred, y_test))

print("\nAccuracy")
print(accuracy_score(y_pred, y_test)*100, '%')


Confusion Matrix
[[ 11   7   1]
 [ 15 110  13]
 [  1  14  39]]

Classification Report
              precision    recall  f1-score   support

           0       0.41      0.58      0.48        19
           1       0.84      0.80      0.82       138
           2       0.74      0.72      0.73        54

    accuracy                           0.76       211
   macro avg       0.66      0.70      0.68       211
weighted avg       0.77      0.76      0.76       211


Accuracy
75.82938388625593 %
